# Imports

In [ ]:
# Import python packages
import warnings
warnings.filterwarnings("ignore")
import streamlit as st

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
from snowflake.core import Root
from snowflake.cortex import complete
session = get_active_session()
root = Root(session)

# 1. Unstructured Data (Cortex Search)
## 1.1 Overview

In [ ]:
SELECT * FROM DIRECTORY('@MARKETING_CAMPAIGNS');

## 1.2 Retrieval Service Creation (PDFs)

In [ ]:
-- Layout extraction for PDF documents (Marketing Campaigns)
CREATE OR REPLACE TABLE _UNSTR_MARKETING_CAMPAIGNS AS
WITH DOCUMENTS AS (
    SELECT 
        RELATIVE_PATH,
        GET_PRESIGNED_URL(@MARKETING_CAMPAIGNS, RELATIVE_PATH, 604800) AS URL,
        SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
            '@MARKETING_CAMPAIGNS',
            RELATIVE_PATH,
            {'mode': 'LAYOUT'}
        ):content::TEXT AS RAW_CONTENT,
        -- Filter for text after first header
        SUBSTRING(RAW_CONTENT, POSITION('#', RAW_CONTENT)) AS DOCUMENT_CONTENT
    FROM 
        DIRECTORY('@MARKETING_CAMPAIGNS')
)
SELECT 
    RELATIVE_PATH,
    URL,
    DOCUMENT_CONTENT
FROM 
    DOCUMENTS;

SELECT * FROM _UNSTR_MARKETING_CAMPAIGNS;

In [ ]:
SELECT * FROM _UNSTR_MARKETING_CAMPAIGNS;

In [ ]:
-- Create a Cortex Search Service (Marketing Campaigns)
CREATE CORTEX SEARCH SERVICE IF NOT EXISTS SEARCH_MARKETING_CAMPAIGNS
  ON DOCUMENT_CONTENT
  ATTRIBUTES RELATIVE_PATH, URL
  WAREHOUSE = AI_WH
  TARGET_LAG = '12 hours'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0-8k'
AS (
  SELECT
      RELATIVE_PATH,
      URL,
      DOCUMENT_CONTENT
  FROM _UNSTR_MARKETING_CAMPAIGNS
);

In [ ]:
-- Layout extraction for PDF documents (News Articles)
CREATE OR REPLACE TABLE _UNSTR_NEWS_ARTICLES AS
WITH DOCUMENTS AS (
    SELECT 
        RELATIVE_PATH,
        GET_PRESIGNED_URL(@NEWS_ARTICLES, RELATIVE_PATH, 604800) AS URL,
        SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
            '@NEWS_ARTICLES',
            RELATIVE_PATH,
            {'mode': 'LAYOUT'}
        ):content::TEXT AS RAW_CONTENT,
        -- Filter for text after first header
        SUBSTRING(RAW_CONTENT, POSITION('#', RAW_CONTENT)) AS DOCUMENT_CONTENT
    FROM 
        DIRECTORY('@NEWS_ARTICLES')
)
SELECT 
    RELATIVE_PATH,
    URL,
    DOCUMENT_CONTENT
FROM 
    DOCUMENTS;

SELECT * FROM _UNSTR_NEWS_ARTICLES;

In [ ]:
-- Create a Cortex Search Service (News Articles)
CREATE CORTEX SEARCH SERVICE IF NOT EXISTS SEARCH_NEWS_ARTICLES
  ON DOCUMENT_CONTENT
  ATTRIBUTES RELATIVE_PATH, URL
  WAREHOUSE = AI_WH
  TARGET_LAG = '12 hours'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0-8k'
AS (
  SELECT
      RELATIVE_PATH,
      URL,
      DOCUMENT_CONTENT
  FROM _UNSTR_NEWS_ARTICLES
);

## 1.3 Test Retrieval Service in RAG pipeline

In [ ]:
# Question
question = 'What products were featured in the stay hydrated lover campaign?'
model = 'mistral-large2'

# Connect to Search Service
search_service = (root
  .databases["AI_DEVELOPMENT"]
  .schemas["SI_THE_FOOD_AND_BEVERAGE_COMPANY"]
  .cortex_search_services["SEARCH_MARKETING_CAMPAIGNS"]
)

# Search relevant documents
search_results = search_service.search(
  query=question,
  columns=["RELATIVE_PATH","DOCUMENT_CONTENT"],
  limit=1
)

retrieved_document = search_results.results[0]['DOCUMENT_CONTENT']

# Generate response with provided context
_ = st.write_stream(complete(model, f"{question} Answer based on this context: {retrieved_document}", stream=True))

with st.expander('Source:'):
    st.markdown(retrieved_document)

## 1.4 AISQL

Not all data is useful for RAG but rather if you turn it into structured data.  
With Snowflake's AISQL functions, you can easily create BI-ready data from unstructured data.

In this example we are transcribing customer reviews from `audio files` and combine it with other reviews already in the `__CUSTOMER_REVIEWS` table.  

In [ ]:
-- Already collected customer reviews
SELECT * FROM __CUSTOMER_REVIEWS LIMIT 5;

## 1.4.1 Transcribe Audio Files

In [ ]:
CREATE OR REPLACE TABLE CUSTOMER_REVIEW_RAW_TRANSCRIPTS AS
WITH TRANSCRIBED_CUSTOMER_REVIES AS (
    -- Transcribe audo files
    SELECT 
        RELATIVE_PATH,
        SPLIT_PART(RELATIVE_PATH,'_',0)::DATE AS REVIEW_DATE,
        TO_NUMBER(TO_CHAR(REVIEW_DATE, 'YYYYMMDD')) AS DATE_KEY,
        TO_FILE('@CUSTOMER_VOICE_REVIEWS', RELATIVE_PATH) AS AUDIO_FILE,
        AI_TRANSCRIBE(AUDIO_FILE) AS TRANSCRIPTION
    FROM
        DIRECTORY('@CUSTOMER_VOICE_REVIEWS')
)
SELECT
    DATE_KEY,
    RELATIVE_PATH,
    TRANSCRIPTION['text']::TEXT AS REVIEW_TEXT,
    TRANSCRIPTION['audio_duration']::FLOAT AS TRANSCRIPTION_DURATION,
FROM TRANSCRIBED_CUSTOMER_REVIES;

SELECT * FROM CUSTOMER_REVIEW_RAW_TRANSCRIPTS LIMIT 10;

## 1.5 Turning Unstructured Data into Structured Data

In [ ]:
CREATE TABLE _SENTIMENTS_CUSTOMER_REVIEWS AS (
    -- Combine transcripts with existing reviews
    WITH COMBINED_DATA AS (
        SELECT DATE_KEY, REVIEW_TEXT FROM __CUSTOMER_REVIEWS
        UNION ALL BY NAME
        SELECT DATE_KEY, REVIEW_TEXT FROM CUSTOMER_REVIEW_RAW_TRANSCRIPTS
    ),
    -- Simple sentiment analysis for 4 different categories
    BASIC_ANALYSIS AS (
        SELECT
            DATE_KEY,
            REVIEW_TEXT,
            LENGTH(REVIEW_TEXT) AS REVIEW_LENGTH,
            AI_SENTIMENT(
              REVIEW_TEXT,
              ['brand', 'product', 'price', 'quality']
            )['categories'] AS SENTIMENT_CATEGORIES
        FROM COMBINED_DATA LIMIT 5
    ),
    -- flatten the returned JSON from AI_SENTIMENT into 1 row per category
    -- also decode the returned values for easier aggregation
    -- finally pivot the rows into multiple columns
    FLATTENED_DATA AS (
        SELECT
            DATE_KEY,
            REVIEW_LENGTH,
            REVIEW_TEXT,
            flattened_data.VALUE['name']::TEXT AS SENTIMENT_CATEGORY,
            DECODE(
              flattened_data.VALUE['sentiment']::TEXT,
              'unknown', NULL,
              'positive', 1,
              'neutral', 0,
              'mixed', 0,
              'negative', -1
            ) AS SENTIMENT_VALUE,
        FROM BASIC_ANALYSIS,
          LATERAL FLATTEN(SENTIMENT_CATEGORIES) flattened_data
    )
    SELECT
        DATE_KEY,
        REVIEW_LENGTH,
        REVIEW_TEXT,
        pivoted_data."'overall'" AS SENTIMENT_OVERALL,
        pivoted_data."'brand'" AS SENTIMENT_BRAND,
        pivoted_data."'price'" AS SENTIMENT_PRICE,
        pivoted_data."'product'" AS SENTIMENT_PRODUCT,
        pivoted_data."'quality'" AS SENTIMENT_QUALITY
    FROM
        FLATTENED_DATA
    PIVOT (
        MAX(SENTIMENT_VALUE) 
        FOR SENTIMENT_CATEGORY IN (ANY ORDER BY SENTIMENT_CATEGORY)
    ) AS pivoted_data
);

SELECT * FROM _SENTIMENTS_CUSTOMER_REVIEWS;

### Matching Reviews to Product Hierarchy via LLMs

In [ ]:
CREATE TABLE _SENTIMENTS_PRODUCTS_CUSTOMER_REVIEWS AS 
-- Join the first hierarchy level to each review
WITH REVIEWS_X_PRODUCT_SUBCATEGORIES AS (
    SELECT
        *
    FROM 
        _SENTIMENTS_CUSTOMER_REVIEWS
    CROSS JOIN
    (
      SELECT 
        ARRAYAGG(DISTINCT CATEGORY_HIER_1_NAME) CATEGORIES
      FROM 
        DIM_PRODUCT_HIERARCHY
    )
),
-- use AI_CLASSIFY to extract hierarchy level 1
PRODUCT_CATEGORIES AS (
    SELECT 
        * EXCLUDE CATEGORIES,
        AI_CLASSIFY(
            REVIEW_TEXT, 
            CATEGORIES,
            {'output_mode': 'single'}
        )['labels'][0]::TEXT AS REVIEW_PRODUCT_CATEGORY,
    FROM 
        REVIEWS_X_PRODUCT_SUBCATEGORIES
        CROSS JOIN
        (
          SELECT
            CATEGORY_HIER_1_NAME,
            ARRAYAGG(DISTINCT CATEGORY_HIER_2_NAME) SUBCATEGORIES
          FROM 
            DIM_PRODUCT_HIERARCHY
          GROUP BY
            CATEGORY_HIER_1_NAME
        )
        WHERE REVIEW_PRODUCT_CATEGORY = CATEGORY_HIER_1_NAME
),
-- Join the second hierarchy level to each review and extract the hierarchy level 2 using AI_CLASSIFY
PRODUCT_SUBCATEGORIES AS (
    SELECT 
        * EXCLUDE CATEGORY_HIER_1_NAME,
        AI_CLASSIFY(
            REVIEW_TEXT, 
            SUBCATEGORIES,
            {
              'output_mode': 'single',
              'examples':[
                {
                  'input': 'I love the Champagne Chocolate Truffles.',
                  'labels': ['Premium Chocolate'],
                  'explanation': 'The review mentions a chocolate that contains champagne as a premium ingredient'
                },
                {
                  'input': 'The Milk Chocolate Classic exceeded all my expectations',
                  'labels': ['Chocolate bars'],
                  'explanation': 'The review does not mention any premium ingredient for the chocolate.'
                }
              ]
            }
        )['labels'][0]::TEXT AS REVIEW_PRODUCT_SUBCATEGORY  
    FROM 
        PRODUCT_CATEGORIES
)
SELECT 
    * EXCLUDE SUBCATEGORIES 
FROM PRODUCT_SUBCATEGORIES;

SELECT * FROM _SENTIMENTS_PRODUCTS_CUSTOMER_REVIEWS;

## 1.5.1 BI Analysis

In [ ]:
CREATE TABLE FACT_CUSTOMER_REVIEWS AS 
SELECT 
    TO_NUMBER(TO_CHAR(DATE_TRUNC('MM',TO_DATE(DATE_KEY::STRING, 'YYYYMMDD')), 'YYYYMMDD')) AS DATE_KEY_START,
    TO_NUMBER(TO_CHAR(LAST_DAY(DATE_TRUNC('MM',TO_DATE(DATE_KEY::STRING, 'YYYYMMDD'))), 'YYYYMMDD')) AS DATE_KEY_END,
    REVIEW_PRODUCT_CATEGORY,
    REVIEW_PRODUCT_SUBCATEGORY,
    AVG(REVIEW_LENGTH) AS AVERAGE_REVIEW_LENGTH,
    AVG(SENTIMENT_OVERALL) AS AVERAGE_SENTIMENT_OVERALL,
    AVG(SENTIMENT_BRAND) AS AVERAGE_SENTIMENT_BRAND,
    AVG(SENTIMENT_PRICE) AS AVERAGE_SENTIMENT_PRICE,
    AVG(SENTIMENT_QUALITY) AS AVERAGE_SENTIMENT_QUALITY
FROM 
    _SENTIMENTS_PRODUCTS_CUSTOMER_REVIEWS
WHERE
    REVIEW_PRODUCT_CATEGORY is not null AND
    REVIEW_PRODUCT_SUBCATEGORY is not null
GROUP BY
    REVIEW_PRODUCT_CATEGORY,
    REVIEW_PRODUCT_SUBCATEGORY,
    DATE_KEY_START,
    DATE_KEY_END
ORDER BY
    REVIEW_PRODUCT_CATEGORY,
    REVIEW_PRODUCT_SUBCATEGORY,
    DATE_KEY_START,
    DATE_KEY_END;

SELECT * FROM FACT_CUSTOMER_REVIEWS;